In [1]:
!pip install -q sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.9 MB/s eta 0:00:00


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd
from bs4 import BeautifulSoup

# Load both datasets
ai_path = "top_ai_questions.json"
ds_path = "top_datascience_questions.json"

ai_df = pd.read_json(ai_path)
ds_df = pd.read_json(ds_path)

# Combine datasets into one
combined_df = pd.concat([ai_df, ds_df], ignore_index=True)

# Remove HTML tags from 'body' using BeautifulSoup
def clean_html(raw_html):
    return BeautifulSoup(raw_html, "html.parser").get_text()

combined_df['clean_body'] = combined_df['body'].apply(clean_html)

# Create single searchable field
combined_df['document_text'] = combined_df['title'] + ". " + combined_df['clean_body']

# Add unique ID
combined_df.reset_index(drop=True, inplace=True)
combined_df['doc_id'] = combined_df.index

# Preview
combined_df[['doc_id', 'title', 'document_text']].head(10)



,doc_id,title,document_text
0,0,Could a paradox kill an AI?,Could a paradox kill an AI?. In Portal 2 we se...
1,1,What is the difference between artificial inte...,What is the difference between artificial inte...
2,2,How could self-driving cars make ethical decis...,How could self-driving cars make ethical decis...
3,3,How can neural networks deal with varying inpu...,How can neural networks deal with varying inpu...
4,4,Do scientists know what is happening inside ar...,Do scientists know what is happening inside ar...
5,5,What is self-supervised learning in machine le...,What is self-supervised learning in machine le...
6,6,What&#39;s the difference between model-free a...,What&#39;s the difference between model-free a...
7,7,How is it possible that deep neural networks a...,How is it possible that deep neural networks a...
8,8,Why does the transformer do better than RNN an...,Why does the transformer do better than RNN an...
9,9,Why do we need explainable AI?,Why do we need explainable AI?. If the origina...


In [4]:
!pip install rank_bm25


In [5]:
import pandas as pd
import re
import numpy as np
from rank_bm25 import BM25Okapi
from bs4 import BeautifulSoup

# --- Clean HTML from text ---
def clean_html(raw_html):
    return BeautifulSoup(raw_html, "html.parser").get_text()

# --- Tokenization (regex-based) ---
def simple_tokenizer(text):
    return re.findall(r'\b\w+\b', text.lower())

# --- Prepare corpus ---QA
rag_df = pd.DataFrame({
    "doc_id": combined_df["doc_id"],
    "text": combined_df["title"].fillna('') + "\n" +
            combined_df["body"].fillna('').apply(clean_html)
})
rag_df["clean_text"] = rag_df["text"].apply(clean_html)
tokenized_corpus = [simple_tokenizer(doc) for doc in rag_df["clean_text"]]

# --- BM25 Index ---
bm25 = BM25Okapi(tokenized_corpus)

# --- BM25 Search Function ---
def search_bm25(query, top_k=10):
    tokens = simple_tokenizer(query)
    scores = bm25.get_scores(tokens)
    top_indices = np.argsort(scores)[::-1][:top_k]

    results = rag_df.iloc[top_indices].copy()
    results["score"] = scores[top_indices]
    return results[["doc_id", "clean_text", "score"]].reset_index(drop=True)

# --- Test it ---
search_bm25("difference between AI and machine learning")


,doc_id,clean_text,score
0,89,What is the difference between machine learnin...,15.865382
1,1446,Difference between machine learning and artifi...,15.607578
2,1149,What is the difference between explainable and...,14.967498
3,961,What is difference between text classification...,14.171515
4,19,What is the difference between strong-AI and w...,12.778757
5,479,What's the difference between Starcraft and Do...,12.765474
6,255,"What does AI software look like, and how is it...",12.735926
7,183,What is the difference between artificial inte...,12.535792
8,1476,Difference between a target and a label in mac...,12.237721
9,608,Which problems in AI are not machine learning?...,12.093825


In [ ]:
!pip install -U sentence-transformers


In [6]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load MiniLM model for sentence embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Precompute corpus embeddings (use the cleaned text)
corpus_embeddings = embedding_model.encode(rag_df['clean_text'].tolist(), convert_to_tensor=False)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
def semantic_search(query, top_k=10):
    query_embedding = embedding_model.encode([query])
    similarities = cosine_similarity(query_embedding, corpus_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_k]

    results = rag_df.iloc[top_indices].copy()
    results["score"] = similarities[top_indices]
    return results[["doc_id", "clean_text", "score"]].reset_index(drop=True)


In [8]:
semantic_search("how to become a data scientist")


,doc_id,clean_text,score
0,1242,How to self-learn data science?\nI am a self-t...,0.730832
1,1418,"I am a programmer, how do I get into field of ...",0.697508
2,1046,"Starting my career as Data Scientist, is Softw...",0.640664
3,886,Do I need to learn Hadoop to be a Data Scienti...,0.576117
4,1084,Data science without knowledge of a specific t...,0.552445
5,986,Data Science Project Ideas\nI don't know if th...,0.512704
6,968,"Books about the ""Science"" in Data Science?\nWh...",0.503817
7,1016,Is Data Science the Same as Data Mining?\nI am...,0.474572
8,786,Data scientist vs machine learning engineer\nW...,0.474530
9,906,What do you think of Data Science certificatio...,0.428231


In [9]:
from sklearn.metrics import ndcg_score
from pprint import pprint

# Step 1–2: Generate evaluation set using both BM25 and MiniLM
sample_queries = combined_df['title'].dropna().sample(10, random_state=42).tolist()
eval_set = []

for query in sample_queries:
    bm25_results = search_bm25(query, top_k=3)

    semantic_raw = semantic_search(query, top_k=3)
    # Handle different return formats safely

    if isinstance(semantic_raw, pd.DataFrame):
        semantic_ids = semantic_raw['doc_id'].tolist()
    elif isinstance(semantic_raw, list):
        semantic_ids = [doc_id for doc_id, _ in semantic_raw]
    else:
        raise ValueError("semantic_search returned unexpected type")

    bm25_ids = bm25_results['doc_id'].tolist()
    relevant_ids = list(set(bm25_ids) | set(semantic_ids))  # union

    eval_set.append({
        "query": query,
        "relevant_doc_ids": relevant_ids
    })

pprint(eval_set)

# --- Evaluation function ---
def evaluate_retriever(retrieval_fn, evaluation_set, top_k=10):
    map_scores, mrr_scores, ndcg_scores = [], [], []

    for sample in evaluation_set:
        query = sample["query"]
        relevant = set(sample["relevant_doc_ids"])

        # Handle both DataFrame and list-of-tuples returns
        results = retrieval_fn(query, top_k=top_k)
        if isinstance(results, list):  # from semantic_search()
            if isinstance(results[0], (list, tuple)):
                retrieved_ids = [doc_id for doc_id, _ in results]
            else:
                retrieved_ids = list(results)
        else:
            retrieved_ids = results["doc_id"].tolist()

        # MAP@10
        num_hits = 0
        avg_precision = 0
        for i, doc_id in enumerate(retrieved_ids):
            if doc_id in relevant:
                num_hits += 1
                avg_precision += num_hits / (i + 1)
        map_scores.append(avg_precision / max(len(relevant), 1))

        # MRR@10
        rr = 0
        for i, doc_id in enumerate(retrieved_ids):
            if doc_id in relevant:
                rr = 1 / (i + 1)
                break
        mrr_scores.append(rr)

        # nDCG@10
        y_true = [1 if doc_id in relevant else 0 for doc_id in retrieved_ids]
        y_score = [1.0 / (i + 1) for i in range(len(retrieved_ids))]
        ndcg_scores.append(ndcg_score([y_true], [y_score]))

    return {
        "MAP@10": sum(map_scores) / len(map_scores),
        "MRR@10": sum(mrr_scores) / len(mrr_scores),
        "nDCG@10": sum(ndcg_scores) / len(ndcg_scores)
    }

# --- Run evaluations ---
print("Evaluating BM25...")
bm25_scores = evaluate_retriever(search_bm25, eval_set)
print("BM25:", bm25_scores)

print("Evaluating MiniLM...")
semantic_scores = evaluate_retriever(semantic_search, eval_set)
print("MiniLM:", semantic_scores)


[{'query': 'Tool to label images for classification',
  'relevant_doc_ids': [1008, 419, 1476, 1179, 1116]},
 {'query': 'Keras Multiple “Softmax” in last layer possible?',
  'relevant_doc_ids': [1314, 1175, 1368, 1372]},
 {'query': 'How do I use GPT-2 to summarise text?',
  'relevant_doc_ids': [1398, 422, 1111, 94, 447]},
 {'query': 'How to handle rectangular images in convolutional neural networks?',
  'relevant_doc_ids': [1442, 538, 43, 413, 1437]},
 {'query': 'What are the main advantages of using declarative programming '
           'languages for building AI?',
  'relevant_doc_ids': [65, 451, 853, 181, 11]},
 {'query': 'How can I transform names in a confidential data set to make it '
           'anonymous, but preserve some of the characteristics of the names?',
  'relevant_doc_ids': [1394, 1445, 1336, 861, 1070]},
 {'query': 'How to annotate text documents with meta-data?',
  'relevant_doc_ids': [961, 1460, 1111, 1063]},
 {'query': 'How to combine backpropagation in neural nets a

In [10]:
def rrf_fusion(bm25_df, semantic_df, k=60):
    """
    Apply Reciprocal Rank Fusion (RRF) to combine ranked results from two retrieval methods.
    """
    bm25_df = bm25_df.reset_index(drop=True)
    semantic_df = semantic_df.reset_index(drop=True)

    # Assign RRF scores
    def rank_dict(df):
        return {row['doc_id']: 1 / (rank + k) for rank, row in df.iterrows()}

    bm25_scores = rank_dict(bm25_df)
    semantic_scores = rank_dict(semantic_df)

    # Combine scores
    combined_scores = {}
    for doc_id in set(bm25_scores.keys()).union(semantic_scores.keys()):
        combined_scores[doc_id] = bm25_scores.get(doc_id, 0) + semantic_scores.get(doc_id, 0)

    # Convert to DataFrame and reattach document metadata
    combined_df_ranked = pd.DataFrame([
        {'doc_id': doc_id, 'rrf_score': score}
        for doc_id, score in sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    ])

    # Join with full document metadata
    return combined_df_ranked.merge(rag_df, on='doc_id')

# Hybrid search function
def hybrid_search(query, top_k=10):
    bm25_results = search_bm25(query, top_k=top_k * 2)  # more candidates
    semantic_results = semantic_search(query, top_k=top_k * 2)
    return rrf_fusion(bm25_results, semantic_results).head(top_k)


In [11]:
hybrid_search("How does GPT-2 summarize text?")


,doc_id,rrf_score,text,clean_text
0,422,0.033333,How do I use GPT-2 to summarise text?\nIn sect...,How do I use GPT-2 to summarise text?\nIn sect...
1,94,0.031099,Why does GPT-2 Exclude the Transformer Encoder...,Why does GPT-2 Exclude the Transformer Encoder...
2,82,0.030886,How does ChatGPT know math?\nChatGPT is a lang...,How does ChatGPT know math?\nChatGPT is a lang...
3,447,0.030835,Are GPT-3.5 series models based on GPT-3?\nIn ...,Are GPT-3.5 series models based on GPT-3?\nIn ...
4,668,0.030550,What is the difference between the positional ...,What is the difference between the positional ...
5,28,0.029274,Meaning of roles in the API of GPT-4/ChatGPT (...,Meaning of roles in the API of GPT-4/ChatGPT (...
6,45,0.028850,How does the (decoder-only) transformer archit...,How does the (decoder-only) transformer archit...
7,162,0.028783,Why does ChatGPT not give the answer text all ...,Why does ChatGPT not give the answer text all ...
8,70,0.028694,What exactly are the &quot;parameters&quot; in...,"What exactly are the ""parameters"" in GPT-3's 1..."
9,177,0.028577,What language is the GPT-3 engine written in?\...,What language is the GPT-3 engine written in?\...


In [12]:
# Evaluate hybrid RRF-based retrieval
hybrid_scores = evaluate_retriever(hybrid_search, eval_set)

print("Hybrid RRF:", hybrid_scores)


Hybrid RRF: {'MAP@10': 0.8503293650793651, 'MRR@10': 1.0, 'nDCG@10': np.float64(0.9485846360899559)}


In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

rag_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=False,
    temperature=0.3
)


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [14]:
def generate_rag_answer(query, top_k=5):
    top_docs = hybrid_search(query, top_k=top_k)
    context_chunks = top_docs['clean_text'].drop_duplicates().tolist()
    truncated_context = ""
    for chunk in context_chunks:
        if len(truncated_context) + len(chunk) < 1500:  # limit context size
            truncated_context += "\n" + chunk
        else:
            break



    prompt = f"""You are an AI assistant. Answer the following question using only the provided context.


Context:
{truncated_context}

Question: {query}
Answer:"""

    response = rag_pipeline(prompt)[0]['generated_text']
    return response


In [16]:
response = generate_rag_answer("how to show text in python?")
print(response)



The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


You are an AI assistant. Answer the following question using only the provided context.


Context:

How do I use GPT-2 to summarise text?
In section 3.6 of the OpenAI GPT-2 paper it mentions summarising text based relates to this, but the method is described in very high-level terms:

To induce summarization behavior we add the text TL;DR: after the article and generate 100 tokens with Top-k random sampling (Fan et al., 2018) with k=2 which reduces repetition and encourages more abstractive summaries than greedy decoding. We use the first 3 generated sentences in these 100 tokens as the summary.

Given a corpus of text, in concrete code terms (python preferred), how would I go about generating a summary of it?


Question: how to show text in python?
Answer:

To show text in Python, you can use the `print()` function. For example, if you want to print the text "Hello World", you can use the following code:

```
print("Hello World")
```

This will print the text on the screen. You can al

In [18]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 126.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.5 MB/s eta 0:00:00


In [23]:
import gradio as gr

def generate_ui_response(query):
    return generate_rag_answer(query).split("Answer:")[-1].strip()

gr.Interface(
    fn=generate_ui_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask your question here...", label="Your Question"),
    outputs=gr.Textbox(label="AI Assistant's Answer"),
    title="💡 RAG Assistant (Hybrid Search + Falcon-7B)",
    description="Ask anything. This assistant retrieves relevant docs and generates an answer using Falcon-7B-Instruct."
).launch(share=True)  # share=True gives you a public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://827f33d24ee6a87bd0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
streamlit run rag_streamlit_app.py


SyntaxError: invalid syntax (<ipython-input-20-391480703>, line 1)